In [1]:
!nvidia-smi

Mon Sep 20 05:40:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Common Setting

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
import sys

COMPETITION_NAME = 'g2net-gravitational-wave-detection'

KAGGLE_DIR = '/content/gdrive/MyDrive/kaggle'
sys.path.append(KAGGLE_DIR)
from scripts.utils import mkdir, load_json

# prepare github
GIT_CONFIG_PATH = f'{KAGGLE_DIR}/secrets/github.json'
GIT_CONFIG = load_json(GIT_CONFIG_PATH)
GIT_USER_NAME = GIT_CONFIG['user.name']
GIT_USER_EMAIL = GIT_CONFIG['user.email']
GIT_TOKEN = GIT_CONFIG['token']
GIT_REPOSITORY_NAME = 'kaggle-' + COMPETITION_NAME
!git config --global user.name {GIT_USER_NAME}
!git config --global user.email {GIT_USER_EMAIL}

# prepare kaggle API
!pip install -q kaggle
!pip install -q --upgrade --force-reinstall --no-deps kaggle
mkdir('/root/.kaggle')
!cp {KAGGLE_DIR}/secrets/kaggle.json /root/.kaggle/


     |████████████████████████████████| 58 kB 5.4 MB/s 
/root/.kaggle was made.


## Prepare Dataset

In [4]:
# prepare input dir
GDRIVE_INPUT_DIR = f'{KAGGLE_DIR}/competitions/{COMPETITION_NAME}/input'
COLAB_INPUT_DIR = '/content/input'
mkdir(GDRIVE_INPUT_DIR)
mkdir(COLAB_INPUT_DIR)


/content/input was made.


In [5]:
# download dataset into gdrive
is_download = False
if is_download:
    !kaggle competitions download -c {COMPETITION_NAME} -p {GDRIVE_INPUT_DIR}


In [12]:
# unzip or copy into colab
is_unzip = False
if is_unzip:
    !unzip -n '{GDRIVE_INPUT_DIR}/{COMPETITION_NAME}.zip' -d {COLAB_INPUT_DIR}

is_copy = True
if is_copy:
    %cp -r {GDRIVE_INPUT_DIR}/* {COLAB_INPUT_DIR}/


In [7]:
# mount gcs to access kaggle personal dataset
is_use_gcs = True
if is_use_gcs:
    # authentication
    from google.colab import auth
    auth.authenticate_user()

    # install gcsfuse
    !echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
    !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
    !apt-get -y -q update
    !apt-get -y -q install gcsfuse

    # mount
    BUCKET1 = "kds-51f153f632521a32be4425a82f5ef3b9d1330dc5cab7403a059157f5" # train part1
    BUCKET2 = "kds-be6b664556db70d53096072ab31ed1183329271fd0df17be418c1cc5" # train part2
    BUCKET3 = "kds-9e101f96a723d803be9e0b1537cbf82c6a27ded057f1df58b1738853" # test

    MOUNT_DIR1 = os.path.join(COLAB_INPUT_DIR, 'train_fold01')
    MOUNT_DIR2 = os.path.join(COLAB_INPUT_DIR, 'train_fold23')
    MOUNT_DIR3 = os.path.join(COLAB_INPUT_DIR, 'test')
    mkdir(MOUNT_DIR1)
    mkdir(MOUNT_DIR2)
    mkdir(MOUNT_DIR3)

    !gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 {BUCKET1} {MOUNT_DIR1}
    !gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 {BUCKET2} {MOUNT_DIR2}
    !gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 {BUCKET3} {MOUNT_DIR3}

deb http://packages.cloud.google.com/apt gcsfuse-bionic main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  90607      0 --:--:-- --:--:-- --:--:-- 90607
OK
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.downlo

## Prepare working directory

In [8]:
# prepare work dir
from scripts.utils import get_work_dir
WORKING_DIR = f'{KAGGLE_DIR}/competitions/{COMPETITION_NAME}/working'
WORK_DIR = get_work_dir(WORKING_DIR)
%cd {WORK_DIR}


/content/gdrive/MyDrive/kaggle/competitions/g2net-gravitational-wave-detection/working/001


## Install Packages

In [9]:
!pip install -r requirements.txt

     |████████████████████████████████| 376 kB 28.5 MB/s 


## Training

In [11]:
# Training
!python train.py -i {COLAB_INPUT_DIR}

========== fold: 0 training ==========
/usr/local/lib/python3.7/dist-packages/nnAudio/utils.py:326: SyntaxWarning: If fmax is given, n_bins will be ignored
  warnings.warn('If fmax is given, n_bins will be ignored',SyntaxWarning)
CQT kernels created, time used = 0.0553 seconds
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b0_ns-c0e6a31c.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b0_ns-c0e6a31c.pth
2021-09-20 05:02:15.104737: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-20 05:02:15.104795: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (421531be15c2): /proc/driver/nvidia/version does not exist
2021-09-20 05:02:16.110525: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 

In [ ]:
# Commit & Push
!git add .
!git commit -m "training finished"
!git push origin main

## Testing

In [13]:
import glob
cps = glob.glob(f"./output/*_best_score.pth")
for c in cps:
    print(c)
cps = ",".join(cps)

./output/tf_efficientnet_b0_ns_fold0_best_score.pth
./output/tf_efficientnet_b0_ns_fold1_best_score.pth
./output/tf_efficientnet_b0_ns_fold2_best_score.pth
./output/tf_efficientnet_b0_ns_fold3_best_score.pth


In [43]:
# Make Submission
!python make_submission.py -i {COLAB_INPUT_DIR} -c {cps}

/usr/local/lib/python3.7/dist-packages/nnAudio/utils.py:326: SyntaxWarning: If fmax is given, n_bins will be ignored
  warnings.warn('If fmax is given, n_bins will be ignored',SyntaxWarning)
CQT kernels created, time used = 0.0112 seconds
2021-09-20 07:04:11.869143: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-20 07:04:11.877666: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-20 07:04:11.878343: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-20 07:04:11.879300: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow bina

In [41]:
# Post Submission
exp_num = os.path.basename(WORK_DIR)
!kaggle competitions submit -c {COMPETITION_NAME} -f ./output/submission.csv -m {exp_num}

100% 4.66M/4.66M [00:05<00:00, 865kB/s]
Successfully submitted to G2Net Gravitational Wave Detection